In [1]:
# Simple RAG for QA about Document
# Sila. Novermber 29th, 2024

In [2]:
import ollama

LLM = "tinyllama"

response = ollama.generate(model=LLM, prompt="Do you know about Claude 3?")

print(response["response"])

Yes, I do! Claude 3 is a popular and powerful virtual assistant software developed by the French company Xamarin. It allows users to control their home automation devices remotely with voice commands through their smartphone or tablet. The software uses deep learning technology to learn the user's voice patterns and respond appropriately, making it easy to use for beginners as well as experienced users. With Claude 3, users can set up smart home scenes, adjust room temperatures, turn on/off lights, start music, and even control appliances like washing machines or refrigerators!


In [3]:
embedding_model = "nomic-embed-text"

embeds = ollama.embeddings(model=embedding_model, prompt="Do you know about Claude 3?")

type(embeds)

ollama._types.EmbeddingsResponse

In [4]:
embeds["embedding"][:5], len(embeds["embedding"])

([0.1646595597267151,
  -0.6364230513572693,
  -4.215642929077148,
  -0.866477906703949,
  -0.22994472086429596],
 768)

In [5]:
import os
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Weaviate

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains.question_answering import load_qa_chain

In [7]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader

In [8]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

In [9]:
filename = r'C:\Users\sila\PycharmProjects\RAG_Supervisor\Success.pdf'

In [10]:
def load_docs(directory):
    loader = PyPDFLoader(directory)
    documents = loader.load()
    return documents

documents = load_docs(filename)

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_documents(documents, chunk_size=500, chunk_overlap=20):
    text_split = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_split.split_documents(documents)
    return chunks

In [12]:
chunks = split_documents(documents)
print(f"Total number of chunks: {len(chunks)}")
print("\n")
embeddings = OllamaEmbeddings(model="nomic-embed-text") # Later, replace with newer langchain code.

Total number of chunks: 299




C:\Users\sila\AppData\Local\Temp\ipykernel_17128\634650621.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text") # Later, replace with newer langchain code.


In [13]:
db = Chroma.from_documents(chunks, embeddings, collection_name = "local-rag")

In [14]:
retriever = db.as_retriever()

In [15]:
template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question:
If you don't know the answer, then answer from your own knowledge and dont give just one word answer, and dont tell the user that you are answering from your knowledge.
Use three sentences maximum and keep the answer concise.

Question: {question}
Context: {context}
Answer:

"""

In [16]:
prompt = ChatPromptTemplate.from_template(template)

In [17]:
local_model = "tinyllama"
llm = ChatOllama(model=local_model)

C:\Users\sila\AppData\Local\Temp\ipykernel_17128\1478007701.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=local_model)


In [18]:
rag_chain = (
        {"context": retriever,  "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
)

In [ ]:
while True:
    query = str(input("Enter Question: "))
    print(rag_chain.invoke(query))

Enter Question:  What will the graduate have knowledge about?


In [19]:
import gradio

C:\Users\sila\PycharmProjects\RAG_Supervisor\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
#a Gradio frontend make sure you have run previous cells
import gradio as gr

iface = gr.Interface(
    fn=rag_chain.invoke,  #use the function we defined in a previous cell
    inputs="text", 
    outputs= "text"
)

iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
